<a href="https://colab.research.google.com/github/gvanathip/Thai-Travel-Open-AI-Chatbot-with-GPT2/blob/main/scrap_traval_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'https://thai.tourismthailand.org/Trip-Planner/search?menu=route&perpage=9'

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
driver.get(url)

In [ ]:
p = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/div/div/div[5]/div/div/div/ul').get_attribute("innerText")

In [ ]:
import time

In [ ]:
plan_lists = []
for i in range(int(p.split('\n')[-1])):
  urlx = url.replace('&perpage=9','')+'&page='+str(i+1)+'&perpage=9'
  driver.get(urlx)
  time.sleep(5)
  #urlxx = requests.get(urlx)
  #soup = BeautifulSoup(urlxx.content, "html.parser")
  print(urlx)
  # for a_href in soup.find_all("a", href=True):
  #     print(a_href["href"])
  lnk = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/div/div/div[4]/div/div/div/div/div').get_attribute("innerHTML")
  #print(urlx)
  elms = lnk.split('><')
  for elm in elms:
    if 'href' in elm:
      #print('https://thai.tourismthailand.org'+elm.split('"')[1])
      plan_lists.append('https://thai.tourismthailand.org'+elm.split('"')[1])
  #print(lnk)

In [ ]:
plan_lists

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
for i in plan_lists[:3]:

  df = []

  urlxx = requests.get(i)
  soup = BeautifulSoup(urlxx.content, "html.parser")

  #driver.get(i)
  lnk_list = dict()
  time.sleep(5)
  for j in soup.find_all('div'):
      if 'href="/Attraction/' in str(j):
        if '<img alt="' in str(j):
          plc_name = str(j).split('href="/Attraction/')[0].split('<img alt="')[1].split('"')[0]
          # print(plc_name)
          lnk = 'https://thai.tourismthailand.org/Attraction/'+str(j).split('href="/Attraction/')[1].split('">อ่านต่อ<')[0]
          # print(lnk)

          if plc_name not in ['Logo Amazing Thailand','catagory icon image']:
            lnk_list[plc_name] = lnk

  print(lnk_list)

In [ ]:
#titles = []
dfs = []

for i in plan_lists:

  df = []

  urlxx = requests.get(i)
  soup = BeautifulSoup(urlxx.content, "html.parser")

  #driver.get(i)
  time.sleep(5)
  #title = driver.find_element_by_xpath('').get_attribute("innerText")
  try:
    title = soup.find_all('h3')[0].string
  except:
    continue
  #titles.append(title)
  #titles.append(title)
  print(title)
  #print(titles)

  span = []
  for j in soup.find_all('span'):
    span.append(j.string)

  plans = []
  for k in span[2:]:
    if 'ภาค' in str(k):
      region = str(k)
    elif 'วัน' in str(k):
      category = str(k)
    elif 'day' in str(k):
      plans.append(str(k))

  places = []
  pics = dict()
  for i in soup.find_all('img'):
    if 'jpeg' in str(i):
      places.append(str(i).split('"')[1])
      pics[str(i).split('"')[1]] = str(i).split('"')[-2]

  lnk_list = dict()
  for j in soup.find_all('div'):
      if 'href="/Attraction/' in str(j):
        if '<img alt="' in str(j):
          try:
            plc_name = str(j).split('href="/Attraction/')[0].split('<img alt="')[1].split('"')[0]
            # print(plc_name)
            lnk = 'https://thai.tourismthailand.org/Attraction/'+str(j).split('href="/Attraction/')[1].split('">อ่านต่อ<')[0]
            # print(lnk)

            if plc_name not in ['Logo Amazing Thailand','catagory icon image']:
              lnk_list[plc_name] = lnk
          except:
            continue

  df.append(title)
  df.append(region)
  df.append(category)
  df.append(plans)
  df.append(places)
  df.append(pics)
  df.append(lnk_list)

  dfs.append(df)
  dfs_ = pd.DataFrame(dfs, columns = ['title','region','category','plan','places','pictures','place_link'])
  print(dfs_)

In [ ]:
dfs_.to_csv('/content/drive/MyDrive/IS-DS/'+'trip_planners.csv',index=False,index_label=False)

In [ ]:
dfs_

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IS-DS/'+'trip_planners.csv')

In [ ]:
set(df.region)

In [ ]:
pd.set_option('max_colwidth', 400)

In [ ]:
df[df.region=='ภาคใต้'].tail(2).place_link